In [55]:
# Importing necessary modules
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv
import asyncio
from IPython.display import display, Markdown

In [56]:
# Load environment variables from a .env file
load_dotenv(override=True)

True

# 🧠 Multi-Agent Research Workflow Using LLMs

This project implements an **automated research pipeline** using an agent-based architecture. It utilizes large language models (LLMs) and web tools to simulate the collaborative workflow of a research team.

---

## 🔄 Workflow Summary

Given a **research query**, the system performs the following:

1. **🗂️ Planner Agent**
   - Decomposes the query into focused search terms.
   - Returns a list of search terms with reasoning.

2. **🔍 Research Agent**
   - Executes real-time web searches using the search terms.
   - Summarizes findings in under 300 words per term.
   - Uses succinct, research-friendly formatting without fluff.

3. **📝 Writer Agent**
   - Synthesizes all search results into a cohesive, structured markdown report.
   - Outputs:
     - A 2-3 sentence summary
     - A full report (1000+ words)
     - Suggested follow-up research questions
          -
## ⚙️ Technical Features

- **Models & Tools**
  - Uses `gpt-4o-mini` with required tool invocation
  - Integrates a `WebSearchTool` for dynamic web data
  - Configured via `ModelSettings`


---


In [57]:
# Instructions for the Planner Agent to generate web search terms
HOW_MANY_SEARCHES = 3 # Number of searches to be performed
INSTRUCTIONS = f"""
You are a helpful research assistant. Given a query, come up with a set of web searches
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for.
"""

# Define the structure for a Web Search Item (search term and its reasoning)
class WebSearchItem(BaseModel):
    reason: str
    """Your reasoning for why this search is important to the query."""

    query: str
    """The search term to use for the web search."""

# Define the structure for a Web Search Plan (list of searches to perform)
class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""

# Initialize the Planner Agent with instructions and expected output type
planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [58]:
# Instructions for the Research Agent
INSTRUCTIONS = """
You are a research assistant. Given a search term, you search the web for that term and
produce a concise summary of the results. The summary must be 2-3 paragraphs and less than 300
words. Capture the main points. Write succinctly, no need to have complete sentences or good
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the
essence and ignore any fluff. Do not include any additional commentary other than the summary itself.
"""

# Initialize the Research Agent with specific tools and settings
research_agent = Agent(
    name="Research agent",  # Agent's name
    instructions=INSTRUCTIONS,  # Agent's instructions
    tools=[WebSearchTool(search_context_size="medium")],  # Tool for web search with low context size
    model="gpt-4o-mini"  # Model to be used
)


In [59]:
# Instructions for the Writer Agent to generate a report
INSTRUCTIONS = """
You are a senior researcher tasked with writing a cohesive report for a research query.
You will be provided with the original query, and some initial research done by a research assistant.
The final output should be in markdown format, and it should be lengthy and detailed.
"""

# Define the structure for the report data (short summary, full markdown report, and follow-up questions)
class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final report"""

    follow_up_questions: list[str]
    """Suggested topics to research further"""

# Initialize the Writer Agent with instructions and expected output type
writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [60]:
# Asynchronous function to plan which searches to perform for a given query
async def plan_searches(query: str):
    """Use the planner_agent to plan which searches to run for the query."""
    print("\n PLANNER AGENT:: Planning searches...")
    # Use the Runner to execute the planner agent with the query
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"\n PLANNER AGENT:: Will perform {len(result.final_output.searches)} searches")
    return result.final_output

# Asynchronous function to perform searches based on the search plan
async def perform_searches(search_plan: WebSearchPlan):
    """Call search() for each item in the search plan."""
    print("\n RESEARCH AGENT :: Started researching...")
    # Create tasks for each search item and execute them concurrently
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("\n RESEARCH AGENT :: Finished researching")
    return results

# Asynchronous function to perform a search using the Research Agent
async def search(item: WebSearchItem):
    """Use the search agent to run a web search for each item in the search plan."""
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    # Run the research agent with the search term and reasoning
    result = await Runner.run(research_agent, input)
    return result.final_output

# Asynchronous function to write a report based on search results
async def write_report(query: str, search_results: list[str]):
    """Use the writer agent to write a report based on the search results."""
    print("\n WRITER AGENT :: Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    # Run the writer agent with the original query and summarized search results
    result = await Runner.run(writer_agent, input)
    print("\n WRITER AGENT :: Completed the report...")
    return result.final_output

In [65]:
# Define the research query to be processed
query = "2025 Agentic usecases in US Healthcare"

# Use the trace context manager to capture the research process
with trace("Research trace"):
    print("\n Starting Research Workflow Orchestration...")
    # Step 1: Plan which searches to perform for the query
    search_plan = await plan_searches(query)
    # Step 2: Perform the searches based on the search plan
    search_results = await perform_searches(search_plan)
    # Step 3: Write a detailed report based on the search results
    report = await write_report(query, search_results)

# https://platform.openai.com/traces


 Starting Research Workflow Orchestration...

 PLANNER AGENT:: Planning searches...

 PLANNER AGENT:: Will perform 3 searches

 RESEARCH AGENT :: Started researching...

 RESEARCH AGENT :: Finished researching

 WRITER AGENT :: Thinking about report...

 WRITER AGENT :: Completed the report...


In [66]:
display(Markdown(report.short_summary))
display(Markdown(report.markdown_report))
follow_up_questions_text = "\n".join(f"- {q}" for q in report.follow_up_questions)
display(Markdown(follow_up_questions_text))

By 2025, agentic AI is poised to fundamentally change U.S. healthcare, focusing on automation in patient care, personalized treatment plans, and operational efficiency. Key applications include advanced patient monitoring, seamless integration with electronic health records, and innovative digital solutions for chronic disease management.

# Agentic AI Use Cases in U.S. Healthcare by 2025

## Introduction
Agentic AI, characterized by its ability to operate autonomously without human intervention, is set to play a crucial role in revolutionizing healthcare delivery in the U.S. by 2025. This report explores the anticipated use cases of agentic AI in various healthcare sectors, its market outlook, and the implications for patient care and operational efficiency.

## Key Applications of Agentic AI in Healthcare
The integration of agentic AI in healthcare focuses on enhancing efficiency and improving patient outcomes through various innovative applications.

### 1. Patient Monitoring
- **Continuous Tracking**: AI agents are expected to utilize wearables and IoT devices to continuously monitor patient vitals. This capability facilitates real-time alerts and allows healthcare providers to implement proactive interventions, significantly improving patient outcomes.

- **Remote Patient Monitoring (RPM)**: RPM technologies will offer healthcare organizations new opportunities to track patient health outside of clinical settings, thereby reducing hospital admissions and supporting a hospital-at-home model for acute care services.

### 2. Personalized Treatment Plans
- **Data-Driven Strategies**: By analyzing genetic, environmental, and lifestyle factors, AI systems will generate personalized treatment strategies tailored to individual patient profiles. This customization aims to enhance treatment effectiveness and minimize adverse side effects.

### 3. Workflow Optimization
- **Administrative Efficiency**: AI's integration into Electronic Health Records (EHRs) is expected to significantly reduce documentation time by up to 50%. This automation will relieve healthcare providers from numerous administrative burdens, allowing them to focus on patient care rather than paperwork.

- **Revenue Cycle Management**: Using AI tools such as VoiceCare's "Joy", healthcare facilities can streamline processes like insurance verification and claims processing, thereby reducing manual workloads and enhancing revenue cycle management.

### 4. Clinical Decision Support
- **Integrated Insights**: AI will enhance clinical decision-making by providing data-driven insights that integrate with EHRs, supporting clinicians in identifying appropriate diagnoses and treatment plans faster and more accurately.

### 5. Digital Therapeutics
- **Chronic Disease Management**: Digital therapeutics emerging as non-invasive solutions can help in the management of chronic diseases and preventive care. These solutions provide patients with accessible tools to manage their health effectively.

### 6. Patient Engagement
- **Predictive Analytics**: AI systems like Actium Health's CENTARI™ can analyze patient data to predict health needs and preferences, significantly improving patient engagement. In pilot projects, this has led to a remarkable 380% increase in wellness visits.

- **Medication Adherence Monitoring**: Workflows powered by AI can monitor patients' medication schedules, sending alerts and reminders to ensure adherence, which has shown to improve rates by 25%.

### 7. Preoperative Support
- **AI-Powered Digital Avatars**: Innovations like the digital avatar developed by Nvidia provide patients with 24/7 access to surgical information, aiming to alleviate anxiety and improve pre-surgical experiences.

## Market Outlook
The market for agentic AI technologies in healthcare is projected to grow significantly, reaching an estimated **$4.96 billion** by 2030. This growth is largely attributed to ongoing advancements in areas such as patient monitoring, personalized healthcare, and workflow optimizations.

The investments in AI technologies are likely to foster an environment where healthcare providers can deliver improved patient-centered care, paving the way for a healthcare landscape that prioritizes efficiency, accessibility, and superior outcomes.

## Challenges and Considerations
While the prospects for agentic AI in healthcare are promising, several challenges need to be addressed:
- **Data Privacy and Cybersecurity**: With increased reliance on AI, the safeguarding of sensitive patient information is paramount. Healthcare organizations must ensure compliance with regulations and implement robust cybersecurity measures to protect against potential breaches.
- **Integration and Interoperability**: As organizations adopt various AI solutions, achieving seamless interoperability across systems to ensure coordinated care will be essential.
- **Acceptance and Trust**: Building trust among healthcare providers and patients in AI-driven solutions is crucial, requiring transparency and demonstrated benefits of AI interventions.

## Conclusion
In conclusion, agentic AI is anticipated to significantly transform U.S. healthcare by 2025 through applications aimed at automating routine tasks and enhancing personalized patient care. The trends indicate a shift towards more efficient healthcare delivery models, though challenges related to data privacy, system integration, and trust will need to be navigated carefully. By leveraging these technologies, the healthcare sector can enhance operational efficacy and outcomes, ultimately leading to a more patient-centric approach to health management.

- What specific AI technologies are being implemented in U.S. healthcare currently?
- How are healthcare providers addressing data privacy concerns with the integration of AI?
- What is the expected impact of AI on healthcare costs by 2025?
- How do patients perceive the use of agentic AI in their care?
- What regulatory frameworks are in place to govern the use of AI in healthcare?